In [1]:
import mysql.connector, time, json, random


alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"


class Statki:
    maxwynik = 10
    
    def __init__(self):
        self.connect()
        self.username = False

    def connect(self):
        self.host = "localhost"
        self.user = "root"
        self.password = ""
        self.database = "statki"
        self.con = mysql.connector.connect(host = self.host, user = self.user, password = self.password, database = self.database)
        self.cursor = self.con.cursor()
    
    def query(self, q):
        self.close()
        self.connect()
        self.cursor.execute(q)
        result = self.cursor.fetchall()
        return result

    def close(self):
        self.cursor.close()
        self.con.close()

    def login(self, l, p):
        result = self.query(f"select id "+
                            f"from users "+
                            f"where username='{l}' and password=md5('{p}')")
        if len(result) > 1:
            print("Database error!")
        if len(result) == 1:
            self.username = l
            self.id = result[0][0]
            return True
        return False

    def register(self, l, p):
        try:
            self.query(f"insert into users (username, password) "+
                       f"values ('{l}', md5('{p}'))")
            self.con.commit()
            self.login(l, p)
            return True
        except:
            return False

    def creategame(self, size):
        self.query(f"insert into gry (rozmiar, wynika, wynikb) "+
                   f"values ('{size}', '0', '0')")
        self.con.commit()

        self.gamesize = size
        result = self.query("select id from gry order by id desc limit 1")[0]

        return result[0]

    def joingame(self, id):
        game = self.query(f"select useraid, userbid, rozmiar from gry where id = '{id}'")
        if len(game) == 0:
            return False

        game = game[0]

        self.gamesize = int(game[2])

        q = "UPDATE gry SET "
        if game[0] == None or game[0] == self.id:
            q += "useraid"
            self.playersign = 'a'
        elif game[1] == None or game[1] == self.id:
            q += "userbid"
            self.playersign = 'b'
        else:
            return False

        q += f" = '{self.id}' WHERE id = '{id}';"
        
        self.query(q)
        self.con.commit()

        self.gameid = id
        
        return True

    def show(self, board):
        for y in range(len(board) + 1):
            for x in range(len(board) + 1):
                if y == 0 and x == 0:
                    print("  ", end=" ")
                    continue
                if y == 0:
                    print(alphabet[x-1], end=" ")
                    continue
                if x == 0:
                    number = str(y)
                    if y < 10:
                        number += " "
                    print(number, end=" ")
                    continue
                    
                print(board[y-1][x-1], end=" ")
            print("")

    def showall(self):
        print("Przeciwnik: ")
        self.show(self.enemyboard)
        print("\nTwoja: ")
        if self.playersign == "a":
            self.show(self.boarda)
        else:
            self.show(self.boardb)
        print("\n")
        

    def fetchgameinfo(self):
        gry = self.query(f"select boarda, boardb, wynika, wynikb from gry where id = '{self.gameid}'")[0]
        progress = self.query(f"select id, graid, atak, trafiony, user from progres "+
                              f"where graid = '{self.gameid}' and user = '{self.playersign.upper()}'")

        self.boarda = json.loads(gry[0])
        self.boardb = json.loads(gry[1])
        self.wynika = gry[2]
        self.wynikb = gry[3]

        self.enemyboard = self.generateemptyboard()
        for line in progress:
            move = self.uncodeattack(line[2])
            if line[3] == 1:
                self.enemyboard[move[0]][move[1]] = "X"
            else:
                self.enemyboard[move[0]][move[1]] = "O"
        

    def uploadboard(self, board):
        q = "UPDATE gry SET "
        if self.playersign == "a":
            q += "boarda"
            self.boarda = board
        elif self.playersign == "b":
            q += "boardb"
            self.boardb = board
        else:
            return False

        q += f" = '{json.dumps(board)}' WHERE id = '{self.gameid}';"

        self.query(q)
        self.con.commit()

        return True

    def generateemptyboard(self):
        board = []

        for y in range(self.gamesize):
            board.append([])
            for x in range(self.gamesize):
                board[y].append(".")

        return board

    def generaterandomboard(self):
        four = 1
        three = 2
        two = 3
        one = 4

        board = self.generateemptyboard()

        for i in range(four):
            self.placerandomship(board, 4)
        for i in range(three):
            self.placerandomship(board, 3)
        for i in range(two):
            self.placerandomship(board, 2)
        for i in range(one):
            self.placerandomship(board, 1)

        return board

    def placerandomship(self, board, segments):
        while True:
            x = random.randint(0, self.gamesize - 1)
            y = random.randint(0, self.gamesize - 1)
            r = random.randint(0, 3)

            if r == 0:
                if x + segments > self.gamesize:
                    continue
            if r == 1:
                if y + segments > self.gamesize:
                    continue
            if r == 2:
                if x - segments < -1:
                    continue
            if r == 3:
                if y - segments < -1:
                    continue

            stop = False
            for i in range(segments + 2):
                for j in range(3):
                    try:
                        if r == 0:
                            if board[y-1+j][x-1+i] != ".":
                                stop = True
                                break
                        if r == 1:
                            if board[y-1+i][x-1+j] != ".":
                                stop = True
                                break
                        if r == 2:
                            if board[y-1+j][x+1-i] != ".":
                                stop = True
                                break
                        if r == 3:
                            if board[y+1-i][x-1+j] != ".":
                                stop = True
                                break
                    except:
                        pass
                if stop:
                    break
            if stop:
                continue

            for i in range(segments):
                if r == 0:
                    board[y][x+i] = str(segments)
                elif r == 1:
                    board[y+i][x] = str(segments)
                elif r == 2:
                    board[y][x-i] = str(segments)
                else:
                    board[y-i][x] = str(segments)

            break

    def zatopiony(self, pos, dir):
        board = self.boarda
        if self.playersign == "a":
            board = self.boardb
        
        if board[pos[0]][pos[1]] != "." and self.enemyboard[pos[0]][pos[1]] != "X":
            return False
        if board[pos[0]][pos[1]] == ".":
            return True
        
        if dir != -1:
            ignore = dir + 2
            if ignore >= 4:
                ignore -= 4
        else:
            ignore = dir
            
        for i in range(4):
            if i == ignore:
                continue

            if i == 0:
                newpos = [pos[0], pos[1] + 1]
            if i == 1:
                newpos = [pos[0] + 1, pos[1]]
            if i == 2:
                newpos = [pos[0], pos[1] - 1]
            if i == 3:
                newpos = [pos[0] - 1, pos[1]]

            if newpos[0] < 0 or newpos[0] >= self.gamesize or newpos[1] < 0 or newpos[1] >= self.gamesize:
                continue
            if not self.zatopiony(newpos, i):
                return False

        return True
                    

    def uncodeattack(self, move):
        x = 0
        for i in range(len(alphabet)):
            if move[0] == alphabet[i]:
                x = i
                break

        y = int(move[1]) - 1

        return [y, x]

    def makemove(self):
        self.showall()
        print("Twoja kolej!\nJesli chcesz wyjsc wpisz QUIT")
        while True:
            move = input("Jaki jest twoj ruch:").upper()
            if move == "QUIT":
                return False
            if len(move) > 2 or len(move) == 0:
                print("Maja byc 2 znaki")
                continue
            if not move[0].isalpha():
                print("Pierwsza ma byc litera")
                continue
            if not move[1].isnumeric():
                print("Druga ma byc liczba")
                continue

            break
        
        attack = self.uncodeattack(move)

        result = False
        if self.playersign == "a":
            if self.boardb[attack[0]][attack[1]] != ".":
                result = True
        else:
            if self.boarda[attack[0]][attack[1]] != ".":
                result = True

        if result:
            print("TRAFIONY!")
            if self.playersign == "a":
                self.wynika += 1
                self.query(f"UPDATE gry SET wynik{self.playersign} = {self.wynika} WHERE id = {self.gameid}")
                self.con.commit()
            elif self.playersign == "b":
                self.wynikb += 1
                self.query(f"UPDATE gry SET wynik{self.playersign} = {self.wynikb} WHERE id = {self.gameid}")
                self.con.commit()
            result = 1
            self.enemyboard[attack[0]][attack[1]] = "X"
            if self.zatopiony(attack, -1):
                print("ZATOPIONY!")
        else:
            print("PUDLO!")
            result = 0
            self.enemyboard[attack[0]][attack[1]] = "O"

        self.query(f"insert into progres (graid, atak, trafiony, user) values ('{self.gameid}', '{move}', '{result}', '{self.id}')")
        self.con.commit()

        self.showall()
        

    def game(self):
        self.fetchgameinfo()

        if self.playersign == "a" and self.boarda == [] or self.playersign == "b" and self.boardb == []:
            while True:
                board = self.generaterandomboard()
    
                print("Generuje losowo plansze dla ciebie...")
                self.show(board)
                choice = input("Czy pasuje ci ta plansza (y/n)")
                if choice.upper() == "N":
                    continue
    
                self.uploadboard(board)
                
                break
        else:
            self.showall()

        while True:
            time.sleep(3)
            
            self.fetchgameinfo()

            if self.playersign == "a":
                if self.wynika >= self.maxwynik:
                    print("WYGRALES!")
                    break
                elif self.wynikb >= self.maxwynik:
                    print("PRZEGRALES!")
                    break
            else:
                if self.wynika >= self.maxwynik:
                    print("PRZEGRALES!")
                    break
                elif self.wynikb >= self.maxwynik:
                    print("WYGRALES!")
                    break

            if self.boarda == [] or self.boardb == []:
                print("Waiting for players to be ready...")
                continue

            progressinfo = self.query(f"select id, graid, atak, trafiony, user from progres where graid = '{self.gameid}' order by id desc limit 1")
            
            if len(progressinfo) == 0 and self.playersign == "a":
                if self.makemove() == False:
                    break
            elif len(progressinfo) > 0:
                if progressinfo[0][4] != self.playersign.upper():
                    if self.makemove() == False:
                        break

            print("Waiting for opponent...")
        
        

statki = Statki()

while True:
    username = input("Podaj username:")
    if username == "":
        break
    password = input("Podaj password:")

    if statki.login(username, password):
        print("Udalo sie zalogowac!")
        break

    print("Niewlasciwe dane!")

if statki.username == False:
    while True:
        print("Rejestracja")
        username = input("Podaj username:")
        if username == "":
            break
        password = input("Podaj password:")

        if statki.register(username, password):
            print("Udalo sie zarejestrowac")
            break

if statki.username != False:
    print("1. Nowa gra\n2. Dolacz do istniejacej gry")

    while True:
        choice = input("Wybierz:")
        try:
            choice = int(choice)
        except:
            print("To nawet nie jest liczba")
            continue

        if choice == 1:
            size = 10
            idgry = statki.creategame(size)
            print("ID twojej gry to: ", idgry)
            if not statki.joingame(idgry):
                print("Nie udalo sie dolaczyc do nowo utworzonej gry")
                continue
            statki.game()
        elif choice == 2:
            idgry = int(input("Podaj id gry:"))
            if not statki.joingame(idgry):
                print("Nie udalo sie dolaczyc do gry")
                continue
            statki.game()
        else:
            print("Nie ma takiego wyboru")
            continue
            
        break

Podaj username: admin
Podaj password: admin


Udalo sie zalogowac!
1. Nowa gra
2. Dolacz do istniejacej gry


Wybierz: 2
Podaj id gry: 1


Waiting for opponent...
Waiting for opponent...
Waiting for opponent...
Waiting for opponent...
Waiting for opponent...
Waiting for opponent...
Waiting for opponent...
Przeciwnik: 
   A B C D E F G H I J 
1  . . . . . . . . . . 
2  . . . . . . . . . . 
3  . . . . . . . . . . 
4  . . . . . . . . . . 
5  . O . . . . . . . . 
6  O X O . . . . . . . 
7  . O . . . . . . . . 
8  . . . . . . . . . . 
9  . . . . . . . . . . 
10 . . . . . . . . . . 

Twoja: 
   A B C D E F G H I J 
1  . . . . . . . 3 3 3 
2  . . . . . 1 . . . . 
3  1 . . . . . . . 2 . 
4  . . . . 1 . . . 2 . 
5  . . . . . . . . . . 
6  . . . 4 4 4 4 . 2 2 
7  . . . . . . . . . . 
8  . . . . . . . 3 3 3 
9  . . . 2 2 . . . . . 
10 . . . . . . . . . 1 


Twoja kolej!
Jesli chcesz wyjsc wpisz QUIT


Jaki jest twoj ruch: a9


TRAFIONY!
ZATOPIONY!
Przeciwnik: 
   A B C D E F G H I J 
1  . . . . . . . . . . 
2  . . . . . . . . . . 
3  . . . . . . . . . . 
4  . . . . . . . . . . 
5  . O . . . . . . . . 
6  O X O . . . . . . . 
7  . O . . . . . . . . 
8  . . . . . . . . . . 
9  X . . . . . . . . . 
10 . . . . . . . . . . 

Twoja: 
   A B C D E F G H I J 
1  . . . . . . . 3 3 3 
2  . . . . . 1 . . . . 
3  1 . . . . . . . 2 . 
4  . . . . 1 . . . 2 . 
5  . . . . . . . . . . 
6  . . . 4 4 4 4 . 2 2 
7  . . . . . . . . . . 
8  . . . . . . . 3 3 3 
9  . . . 2 2 . . . . . 
10 . . . . . . . . . 1 


Waiting for opponent...
Waiting for opponent...
Waiting for opponent...
Waiting for opponent...


KeyboardInterrupt: 